<a href="https://colab.research.google.com/github/yvanminyem/whyzzerRec/blob/main/WhyzzerRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 4.0 MB 14.5 MB/s 


importing different necessary packages

In [2]:
import os
import tempfile

from google.colab import files

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import keras
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

plt.style.use('seaborn-whitegrid')

Access bigquery data from collab

In [3]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  with open('/content/' + fn, 'wb') as f:
    f.write(uploaded[fn])
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/content/' + fn
  projectID = fn.rsplit("-", 1)[0]

Saving whyzzerrecommender-dc1a91d72a11.json to whyzzerrecommender-dc1a91d72a11.json
User uploaded file "whyzzerrecommender-dc1a91d72a11.json" with length 2323 bytes


Enable bigquery Ipython magic

In [4]:
%reload_ext google.cloud.bigquery

extract the first 10 fields

In [5]:
%%bigquery analytics_test_import
SELECT
    *
FROM `whyzzer.kaggledata`


In [6]:
analytics_test_import

,position,channel_id,channel_title,video_id,published_at,video_title,video_description,video_category_id,video_category_label,duration,duration_sec,dimension,definition,caption,licensed_content,view_count,like_count,dislike_count,favorite_count,comment_count
0,4,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,8WlhQ1ouWkU,2020-06-23 19:54:41+00:00,Getting Curious: What it takes to build a TPU ...,Have you ever wondered what kind of background...,28,Science & Technology,PT7M18S,438,2d,hd,True,None,637,28,0,0,0
1,34,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,1WB8DSnV6bY,2020-01-06 15:54:27+00:00,How to use Kernels and Forums to Win Kaggle Co...,Kaggle Days China edition was held on October ...,28,Science & Technology,PT17M6S,1026,2d,hd,False,None,1418,28,0,0,0
2,40,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,LDrhLfl012w,2019-12-20 18:55:30+00:00,Kaggle Live Coding: Fuzzy Matching for Spellin...,This week we'll be working on using fuzzy matc...,28,Science & Technology,PT1H12M24S,744,2d,hd,False,None,2709,86,0,0,0
3,44,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,LpRbOzNshNA,2019-12-11 18:41:46+00:00,Kaggle Reading Group: On NMT Search Errors and...,"This week we'll be starting a new paper: ""On N...",28,Science & Technology,PT1H4M58S,298,2d,hd,False,None,979,36,0,0,0
4,63,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,kDzEZihQFig,2019-10-11 18:00:52+00:00,Kaggle Live Coding: Automating report generati...,This week Rachael will continue to work on her...,28,Science & Technology,PT1H20M56S,1256,2d,hd,False,None,1976,69,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,127,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,8lniZVqRLA0,2019-07-11 00:06:04+00:00,Interview with Abhishek Thakur | World's First...,Help us congratulate Kaggle's very first Tripl...,28,Science & Technology,PT15M25S,925,2d,hd,False,None,52951,1587,19,0,61
312,104,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,GJBOMWpLpTQ,2019-08-12 17:00:00+00:00,How to Enter a Kaggle Competition (using Kerne...,Ever wanted to try out Kaggle competitions but...,28,Science & Technology,PT7M10S,430,2d,hd,False,None,50574,999,20,0,33
313,68,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,8yZMXCaFshs,2019-09-30 18:05:19+00:00,How to Get Started with Kaggle’s Titanic Compe...,New to Kaggle? Our Titanic competition is a gr...,28,Science & Technology,PT6M37S,397,2d,hd,False,None,126383,2630,40,0,61
314,109,UCSNeZleDn9c74yQc-EKnVTA,Kaggle,uWVR_axaVwk,2019-08-06 21:31:29+00:00,Approaching (almost) Any Machine Learning Prob...,Abhishek Thakur is the chief data scientist at...,28,Science & Technology,PT45M54S,2754,2d,hd,False,None,18889,550,44,0,41


In [7]:
analytics_test_import.columns

Index(['position', 'channel_id', 'channel_title', 'video_id', 'published_at',
       'video_title', 'video_description', 'video_category_id',
       'video_category_label', 'duration', 'duration_sec', 'dimension',
       'definition', 'caption', 'licensed_content', 'view_count', 'like_count',
       'dislike_count', 'favorite_count', 'comment_count'],
      dtype='object')

get just the columns we need and save it into one variable

In [8]:
%%bigquery analytics_data_real
SELECT
    video_title,position,published_at
FROM `whyzzer.kaggledata`

In [9]:
analytics_data_real.head()

,video_title,position,published_at
0,Getting Curious: What it takes to build a TPU ...,4,2020-06-23 19:54:41+00:00
1,How to use Kernels and Forums to Win Kaggle Co...,34,2020-01-06 15:54:27+00:00
2,Kaggle Live Coding: Fuzzy Matching for Spellin...,40,2019-12-20 18:55:30+00:00
3,Kaggle Reading Group: On NMT Search Errors and...,44,2019-12-11 18:41:46+00:00
4,Kaggle Live Coding: Automating report generati...,63,2019-10-11 18:00:52+00:00


In [10]:
analytics = analytics_data_real

rename columns to match model entries

In [14]:
analytics.rename(columns={'video_title': 'movie_title', 'published_at': 'timestamp','position':'user_id'}, inplace=True)


Now we load our movielens model

In [21]:
MODEL_PATH ='/content/drive/MyDrive/Colab Notebooks/1629150301'
model = tf.keras.models.load_model(MODEL_PATH )

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [22]:
model.summary()

Model: "brute_force"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
query_model (QueryModel)     multiple                  69548     
_________________________________________________________________
retrieval_1 (Retrieval)      multiple                  0         
Total params: 125,054
Trainable params: 68,544
Non-trainable params: 56,510
_________________________________________________________________
